<a href="https://www.meetup.com/Amazon-Web-Services-Mexico/"><img src="images/AWSCDMX.png"></a>
# MeetUp Grupo de Usuarios AWS CDMX 20190424

# Uso de AWS con Python

## David Jonathan Sol Llaven
T-Systems México

Contacto:
- https://medium.com/@soldavid
- Grupo de Usuarios AWS Ciudad de México: https://www.meetup.com/Amazon-Web-Services-Mexico/
- Canal de Slack: https://launchpass.com/awscdmx

## ¿Cómo manejamos AWS?

### A través de la [Consola AWS](https://aws.amazon.com/console/)

<img src="images/consola.png">

### A través de llamadas API directas

    https://ec2.amazonaws.com/?Action=RunInstances
    &ImageId=ami-2bb65342
    &MaxCount=3
    &MinCount=1
    &Placement.AvailabilityZone=us-east-1a
    &Monitoring.Enabled=true
    &Version=2016-11-15
    &X-Amz-Algorithm=AWS4-HMAC-SHA256
    &X-Amz-Credential=AKIAIOSFODNN7EXAMPLEus-east-1%2Fec2%2Faws4_request
    &X-Amz-Date=20130813T150206Z
    &X-Amz-SignedHeaders=content-type%3Bhost%3Bx-amz-date
    &X-Amz-Signature=ced6826de92d2bdeed8f846f0bf508e8559e98e4b0194b84example54174deb456c
    Content-type: application/json
    host:ec2.amazonaws.com

### A través de [AWS CLI](https://aws.amazon.com/cli/)

``` bash
& aws s3 ls  --profile EJEMPLO
    2018-07-17 15:39:39 cubeta-1
    2018-09-20 21:03:54 cubeta-2
    2018-09-06 11:10:39 cubeta-3
    2018-11-22 17:58:51 bitacoras
    2018-05-21 15:13:49 cf-templates-xxxxxxxxxxxx-us-east-1
    2019-03-26 11:45:38 orquestacion
    2019-02-26 16:04:42 red-hat-openshift-openshiftstack-1
    2019-02-26 16:04:43 red-hat-openshift-openshiftstack-2
    2019-02-26 16:04:50 red-hat-openshift-openshiftstack-3
```

### Ejemplo Script

```bash
#!/bin/bash
# Obtiene los requests de la hora pasada
# David Sol - Noviembre 2018

horaInicialUTC=$(date +%FT%H:00:00Z --date '1 hour ago' -u)
horaFinalUTC=$(date +%FT%H:00:00Z -u)
horaLocal=$(date +%FT%H:30:00Z)

printf "Hora Inicial: %s\n" ${horaInicialUTC}
printf "Hora Final: %s\n" ${horaFinalUTC}

requests=$(aws cloudwatch get-metric-statistics \
    --start-time ${horaInicialUTC} \
    --end-time ${horaFinalUTC} \
    --cli-input-json file://requests-no-hour.json \
    --query 'Datapoints[0].Sum' \
    --output text \
    --profile EJEMPLO)

LC_NUMERIC=en_US.UTF-8
printf -v n_requests "%'.0f" ${requests}
printf "Requests: %s\n" ${n_requests}

sed "s/\[numero\]/${n_requests}/g" base_email.html > current_email.html

mail -a 'Content-Type: text/html' \
     -s "Hits CloudFront corte de las ${horaLocal:11:2}:30" \
     destino@dominio.com \
     < current_email.html
```

### A través de los [SDKs](https://aws.amazon.com/tools/)

- Javascript
- PHP
- Go
- Ruby
- Java
- Muchos más...

### El SDK de Python se llama [Boto3](https://aws.amazon.com/sdk-for-python/)

Nos permite el acceso a los recursos de AWS de forma simple, a traves de:

- Clientes: Acceso de Bajo Nivel

      sqs = boto3.client('sqs')

- Recursos: Acceso de Alto Nivel

      s3 = boto3.resource('s3')

Y nos proporciona mecanismos para ejecutar comandos como son:

- Sesiones

      session = boto3.session.Session()
    
- Colecciones

      for bucket in s3.buckets.all():
          print(bucket.name)
    
- Paginadores

      paginator = client.get_paginator('list_objects')
      page_iterator = paginator.paginate(Bucket='my-bucket')
      for page in page_iterator:
          print(page['Contents'])
        
- Waiters

      waiter = client.get_waiter('alarm_exists')

## Ejemplos en la vida real

No van a funcionar para otras personas, ya que utilizan mi configuración local.

Lo cual es excelente, por que se puede compatir código y documentación sin arriesgar nuestras credenciales.

In [ ]:
# Importa el SDK para AWS
import boto3
# Abre una sesion con un nombre de perfil definido en el archivo ~/.aws/credentials
session = boto3.session.Session(profile_name='DEV')
# También se puede hacer utilizando variables de ambiente del sistema operativo:
# AWS_PROFILE
# AWS_ACCESS_KEY_ID
# AWS_SECRET_ACCESS_KEY
# Para mas información ver https://boto3.amazonaws.com/v1/documentation/api/latest/guide/configuration.html
# Usando esa sesión se abre una conexión tipo recurso a S3
s3 = session.resource('s3')
# Se obtienen todos los buckets en esa cuenta, y se imprime el nombre de cada una de ellas
for bucket in s3.buckets.all():
    print(bucket.name)

In [ ]:
bucket = s3.Bucket('amcfmdevavpc01g')
fecha = bucket.creation_date.strftime('%Y-%m-%d a las %H:%M')
print(f'Bucket {bucket.name} creada el {fecha}')

In [ ]:
import pprint
s3_client = session.client('s3')
paginator = s3_client.get_paginator('list_objects')
page_iterator = paginator.paginate(Bucket='amneedevstgw01a')
for page in page_iterator:
          pprint.pprint(page['Contents'])

In [ ]:
session = boto3.session.Session(profile_name='PRD')
cloudwatch_client = session.client('cloudwatch')
cadena_widget = '{ "metrics": [ [ "AWS/CloudFront", "Requests", "Region", "Global", "DistributionId", "E4MSYTJKDY01N", { "period": 3600, "stat": "Sum" } ] ], "view": "timeSeries", "stacked": false, "width": 800, "height": 300, "start": "-P28D", "end": "P0D", "timezone": "-0500" }'
respuesta = cloudwatch_client.get_metric_widget_image(MetricWidget=cadena_widget)

In [ ]:
respuesta["MetricWidgetImage"]

In [ ]:
from IPython.display import Image
Image(respuesta["MetricWidgetImage"]) 

![Python](images/Python.jpg)
## ¿Qué es [Python](https://www.python.org/)?
Python es un lenguaje:
* Interpretado.
* De tipos de variables dinámico.
* Fácil de aprender.
* Código muy claro.
* Amplio número de librerias para muchos casos de uso.
* Creado en 1991 por Guido van Rossum.
* https://www.python.org/

### PEP 20 - The Zen of Python
#### https://www.python.org/dev/peps/pep-0020/

In [ ]:
import this

### Una historia de dos Pythons

* Python 2 se liberó en 2000
    * Su última versión es 2.7
    * Fuera de soporte en Enero de 2020


* Python 3 se liberó en 2008
    * Su versión actual es 3.7, pronto 3.8
    * Es la versión soportada actualmente


* Python 2 todavía es muy utilizada
    * Mucho software existente aún la utiliza


* Debemos utilizar Python 3 en la medida de lo posible
    * Sobre todo para cosas nuevas

### Una historia de dos Pythons

``` bash
$ python2 --version
Python 2.7.15+
```

``` bash
$ python2 -m pip --version
pip 9.0.1 from /usr/lib/python2.7/dist-packages (python 2.7)
```

``` bash
$ python3 --version
Python 3.6.8
```

``` bash
$ python3 -m pip --version
pip 19.0.3 from /home/corsario/.local/lib/python3.6/site-packages/pip (python 3.6)
```

```bash
$ python --version
Python 2.7.15+
```

### ¿Hay una forma mas simple?

![Meta](images/Goal.png)

#### ¡Si la hay! Ambientes Virtuales.

### Ambientes Virtuales

Nos permiten tener "ambientes" separados cada uno con su propia versión de Python y packetes distintos.

Otra ventaja es que evitan "colisiones" entre los paquetes que requieren diferentes programas.

* VENV - https://docs.python.org/3/library/venv.html

 * Forma estandárd y actualmente soportada.


* VirtualENV - https://virtualenv.pypa.io/en/stable/

 * La forma anterior, posiblemente la más utilizada.

![Anaconda](images/Anaconda.png)
## ¿Qué es [Anaconda](https://www.anaconda.com/)?
Es un plataforma para realizar Ciencia de Datos con Python/R (y mas).
![Anaconda distribution](images/Anaconda_Distribution.png)
* https://www.anaconda.com/

![Conda](images/Conda.svg)
## ¿Qué es [Conda](https://conda.io/en/latest/)?
Es un administrador de __paquetes (librerias), dependencias y ambientes virtuales__ para Python, pero además soporta *cualquier* lenguaje.

Funciona en Linux, MacOs y Windows. No requiere permisos de administrador.

Se puede obtener en dos distribuciones:

- Anaconda: Incluye todos los paquetes de Anaconda, y ocupa alrededor de 3 GB.
- Miniconda: Solo lo minimo requerido, ocupa 400 MB. *Permite instalar paquetes posteriormente*

Mi recomendación es instalar Miniconda, y agregar los paquetes deseados en Ambientes Virtuales. Excepto que sepan y utilicen todo lo que Anaconda incluye.

* https://conda.io/en/latest/

* [Instrucciones de Instalación](https://conda.io/projects/conda/en/latest/user-guide/install/index.html)

* [Conda Cheetsheat](https://docs.conda.io/projects/conda/en/latest/_downloads/1f5ecf5a87b1c1a8aaf5a7ab8a7a0ff7/conda-cheatsheet.pdf)

### Instalar en Windows

[Instrucciones](https://conda.io/projects/conda/en/latest/user-guide/install/windows.html).

1. Desgargar el instalador de Miniconda: [Miniconda Windows 64 bits](https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe).


2. Ejecutarlo (No instalarlo como administrador).


3. Probarlo.

![Anaconda en Windows](images/AnacondaInWindows.png)

In [ ]:
!conda list

### Instalar en MacOS

[Instrucciones](https://conda.io/projects/conda/en/latest/user-guide/install/macos.html).

1. Descargar el script de instalación.

```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-x86_64.sh
```

2. Ejecutar el script.

```bash
bash Miniconda3-latest-MacOSX-x86_64.sh
```

3. Probarlo (en una terminal nueva).

In [ ]:
!conda list

### Instalar en Linux

[Instrucciones](https://conda.io/projects/conda/en/latest/user-guide/install/linux.html).

1. Descargar el script de instalación.

```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
```

2. Ejecutar el script.

```bash
bash Miniconda3-latest-Linux-x86_64.sh
```

3. Probarlo (en una terminal nueva).

In [ ]:
!conda list

## Agregar el canal [conda-forge](https://conda-forge.org/)

Es una colección de paquetes creados por la comunidad para Conda.

Nos da acceso a mas paquetes (incluyendo muchos de AWS) y versiones mas actualizadas de los mismos.

Es menos estable.

``` bash
conda config --add channels conda-forge
```

## [Configurar Conda](https://conda.io/projects/conda/en/latest/user-guide/configuration/use-condarc.html)

Archivo de configuración:

* Linux: /home/<USER>/.condarc
* Windows: C:\Users\<USER>\.condarc

``` YAML
always_yes: True
show_channel_urls: True
channel_priority: strict
channels:
- conda-forge
- defaults
```

Nota: Esta es mi configuración, caveat emptor

## Actualización de Conda
```shell
D:\conda update conda
Collecting package metadata: done
Solving environment: done

## Package Plan ##

  environment location: d:\anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blas-1.0                   |              mkl           6 KB
    libblas-3.8.0              |            6_mkl         3.5 MB  conda-forge
    libcblas-3.8.0             |            6_mkl         3.5 MB  conda-forge
    liblapack-3.8.0            |            6_mkl         3.5 MB  conda-forge
    mkl-service-2.0.1          |   py37hfa6e2cd_0          58 KB  conda-forge
    spyder-3.3.4               |           py37_0         2.7 MB
    ------------------------------------------------------------
                                           Total:        13.3 MB

The following packages will be UPDATED:

  blas                                          conda-forge --> pkgs/main
  cryptography                           2.5-py37h74b6da3_1 --> 2.6.1-py37hb32ad35_0
  icu                          conda-forge::icu-58.2-vc14_0 --> pkgs/main::icu-58.2-ha66f8fd_1
  krb5                                 1.16.3-h038dc86_1000 --> 1.16.3-hdd46e55_1001
  libarchive                            3.3.3-hd1ea8e3_1002 --> 3.3.3-h0005e80_1004
  libblas                           3.8.0-5_h8933c1f_netlib --> 3.8.0-6_mkl
  libcblas                          3.8.0-5_h8933c1f_netlib --> 3.8.0-6_mkl
  liblapack                         3.8.0-5_h8933c1f_netlib --> 3.8.0-6_mkl
  libssh2                               1.8.0-hc4dcbb0_1003 --> 1.8.2-h642c060_2
  lz4-c                                      1.8.1.2-vc14_0 --> 1.8.3-he025d50_1001
  mkl                                            2018.0.3-1 --> 2019.3-203
  mkl-service        pkgs/main::mkl-service-1.1.2-py37hb21~ --> conda-forge::mkl-service-2.0.1-py37hfa6e2cd_0
  mkl_fft            pkgs/main::mkl_fft-1.0.6-py37hdbbee80~ --> conda-forge::mkl_fft-1.0.12-py37hfa6e2cd_0
  mkl_random         pkgs/main::mkl_random-1.0.1-py37h77b8~ --> conda-forge::mkl_random-1.0.2-py37h830ac7b_2
  numpy-base                          1.14.5-py37h4a99626_4 --> 1.16.3-py37hc3f5095_0
  openssl                                 1.0.2r-hfa6e2cd_0 --> 1.1.1b-hfa6e2cd_2
  pyqt               conda-forge::pyqt-5.6.0-py37h764d66f_~ --> pkgs/main::pyqt-5.9.2-py37h6538335_2
  qt                       conda-forge::qt-5.6.2-h2639256_8 --> pkgs/main::qt-5.9.7-vc14h73c81de_0
  scipy                                1.1.0-py37h4f6bf74_1 --> 1.2.1-py37h29ff71c_0
  sip                              4.18.1-py37h6538335_1000 --> 4.19.8-py37h6538335_1000
  vc                                   conda-forge::vc-14-0 --> pkgs/main::vc-14.1-h0510ff6_4
  vs2015_runtime     conda-forge::vs2015_runtime-14.0.2542~ --> pkgs/main::vs2015_runtime-14.15.26706-h3a45250_0

The following packages will be SUPERSEDED by a higher-priority channel:

  numpy              conda-forge::numpy-1.16.3-py37h873a0b~ --> pkgs/main::numpy-1.16.3-py37h19fb1c0_0
  spyder                                        conda-forge --> pkgs/main

The following packages will be DOWNGRADED:

  pycurl                            7.43.0.2-py37h74b6da3_0 --> 7.43.0.2-py37h7a1dbc1_0


Proceed ([y]/n)? y


Downloading and Extracting Packages
libcblas-3.8.0       | 3.5 MB    | ################################################################################# | 100%
mkl-service-2.0.1    | 58 KB     | ################################################################################# | 100%
liblapack-3.8.0      | 3.5 MB    | ################################################################################# | 100%
libblas-3.8.0        | 3.5 MB    | ################################################################################# | 100%
blas-1.0             | 6 KB      | ################################################################################# | 100%
spyder-3.3.4         | 2.7 MB    | ################################################################################# | 100%
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

D:\>conda update conda
Collecting package metadata: done
Solving environment: done

# All requested packages already installed.
```

## Crear ambientes virtuales

Nos permiten desarrollar y/o operar con la tranquilidad de no tener conflictos entre diferentes versiones requeridas de paquetes.

Podemos elegir la versión de Python y los paquetes a utilizar de forma independiente.

In [ ]:
!conda create --name aws python=3.7

### Utilizar el ambiente virtual

In [ ]:
!conda activate aws

## Instalar el SDK de AWS para Python - Boto3

In [ ]:
!conda install awscli boto3 -y

### También podemos instalar AWS CLI

In [ ]:
!conda install awscli -y

### AWS Access Key

Se utilizan en lugar de un usuario y una contraseña para scripts y programas con AWS CLI y SDKs (como Boto3).

![Access Key en la consola](images/AccessKey.png)

![AWS CLI](images/AWSCLI.png)
## Configurar AWS CLI
``` shell
(aws) D:\>aws configure --profile monitor
AWS Access Key ID [****************XXXX]: ****************XXXX
AWS Secret Access Key [****************XXXX]: ****************XXXX
Default region name [us-east-1]: us-west-1
Default output format [json]: json
```

In [ ]:
!aws s3 ls --profile monitor

![Boto3](images/AWSSDK.png)
## Validar Boto3

In [ ]:
import boto3
session = boto3.session.Session(profile_name='monitor')
s3 = session.resource('s3')
for bucket in s3.buckets.all():
    print(bucket.name)

![Jupyter](images/Jupyter.png)
## ¿Qué es [Jupyter Notebook](https://jupyter.org/)?
Es una aplicación web que permite crear y compartir documentos que contienen código real, visualizaciones y texto.

Se puede utilizar para documentar procesos, desarrollar programas interactivamente e incluso dar presentaciones.

## Instalar Jupyter Notebook

In [ ]:
!conda install jupyter -y

## Ejecutar Jupyter Notebook

```bash
$ jupyter notebook
```

![Jupyter home](images/JupyterHome.png)

## [Visual Studio Code](https://code.visualstudio.com/)

Ambiente de Desarrollo Integrado para Python.

![Visual Studio Code](images/VSCode.png)

https://code.visualstudio.com/

### Soporte para Python:

https://code.visualstudio.com/docs/languages/python

## Python Extension

https://marketplace.visualstudio.com/items?itemName=ms-python.python

![Extension](images/Extension.png)

## Desactivar Ambiente Virtual

In [ ]:
!conda deactivate

## Gracias
![Ciencia](images/Science.jpg)

<a style="background-color:black;color:white;text-decoration:none;padding:4px 6px;font-family:-apple-system, BlinkMacSystemFont, &quot;San Francisco&quot;, &quot;Helvetica Neue&quot;, Helvetica, Ubuntu, Roboto, Noto, &quot;Segoe UI&quot;, Arial, sans-serif;font-size:12px;font-weight:bold;line-height:1.2;display:inline-block;border-radius:3px" href="https://unsplash.com/@markusspiske?utm_medium=referral&amp;utm_campaign=photographer-credit&amp;utm_content=creditBadge" target="_blank" rel="noopener noreferrer" title="Download free do whatever you want high-resolution photos from Markus Spiske"><span style="display:inline-block;padding:2px 3px"><svg xmlns="http://www.w3.org/2000/svg" style="height:12px;width:auto;position:relative;vertical-align:middle;top:-2px;fill:white" viewBox="0 0 32 32"><title>unsplash-logo</title><path d="M10 9V0h12v9H10zm12 5h10v18H0V14h10v9h12v-9z"></path></svg></span><span style="display:inline-block;padding:2px 3px">Photo by Markus Spiske on Unsplash</span></a>